In [274]:
import numpy as np
import cmath

In [275]:
class platform:
    def __init__(self,d,D,l1,l2,max_limb_error,max_limb_try_count,max_joint_angle):
        "d: distancia de cada junta universal en la plataforma al centro"
        
        "D: distancia del eje de cada motor al centro de la base"
        
        "la altura del eje del motor es considerada el plano z=0"
        
        "l1: long del antebrazo"
        
        "l2: long del brazo"
        
        "max_limb_error: máximo error (en las mismas unidades de l1 y l2) que es permitido para posicionar el brazo"
        
        "max_limb_try_count: máxima cantidad de iteraciones para calcular el ángulo de un motor, si la cantidad"
        "Si la cantidad requerida de iteraciones es mayor, genera un error llamado 'Point unreachable' "
        
        "max_joint_angle:angulo máximo que puede tener la junta universal, una posición que lo exceda genera un error"
        "llamado 'Joint angle exceeded'"
        
        self.c1=np.array([0,-d,0])
        self.c2=np.array([-d*np.sin(np.pi/3),d*np.cos(np.pi/3),0])
        self.c3=np.array([d*np.sin(np.pi/3),d*np.cos(np.pi/3),0])
        
        self.d=d
        self.D=D

        self.l1=l1
        self.l2=l2
        
        self.max_limb_error=max_limb_error
        self.max_limb_try_count=max_limb_try_count
        
        #defino los angulos de los motores
        self.ang1=0
        self.ang2=0
        self.ang3=0
        #defino los angulos máximos que se les permite rotar a los motores
        self.angle_max=90
        self.angle_min=-90
        #defino el ángulo máximo permitido en las juntas universales
        self.max_joint_angle=max_joint_angle
    def x_rotation_matrix(self,r_x):
        return np.array([[1,0,0],[0,np.cos(r_x),-np.sin(r_x)],[0,np.sin(r_x),np.cos(r_x)]]) 
    
    def y_rotation_matrix(self,r_y):
        return np.array([[np.cos(r_y),0,np.sin(r_y)],[0,1,0],[-np.sin(r_y),0,np.cos(r_y)]])
          
    def rotate(self,movement,joint,angle):
        if movement=="pitch":
            M_r=self.x_rotation_matrix(angle)
        if movement=="roll":
            M_r=self.y_rotation_matrix(angle)
            
        if joint=="C1":
            wm=np.identity(3)
        if joint=="C2":
            wm=self.get_m2_basis()
        if joint=="C3":
            wm=self.get_m3_basis()
        #transformamos los puntos a la base conveniente
        self.c1=np.linalg.solve(wm,self.c1)
        self.c2=np.linalg.solve(wm,self.c2)
        self.c3=np.linalg.solve(wm,self.c3)
        #realizamos la rotación de los mismos
        self.c1=M_r.dot(self.c1)
        self.c2=M_r.dot(self.c2)
        self.c3=M_r.dot(self.c3)
        #corregimos el offset de la plataforma
        if joint=="C1":
            self.apply_offset(self.c2)
        if joint=="C2":
            self.apply_offset(self.c3)
        if joint=="C3":
            self.apply_offset(self.c1)
        #obtengo los puntos en la base original
        self.c1=wm.dot(self.c1)
        self.c2=wm.dot(self.c2)  
        self.c3=wm.dot(self.c3)
        
    def apply_offset (self,pos):
        "la función desplaza la plataforma el offset correspondiente para compenzar el introducido por la rotación"
        allowed_pos=-pos[0]*np.tan(np.pi/6)
        offset=np.array([0,allowed_pos-pos[1],0])
        self.c1=self.c1+offset
        self.c2=self.c2+offset
        self.c3=self.c3+offset
        
    def get_m2_basis(self):
        Wm2=np.array([[-np.cos(np.pi/3),np.cos(np.pi/6),0],[-np.sin(np.pi/3),-np.sin(np.pi/6),0],[0,0,1]])
        return Wm2

    def get_m3_basis(self):
        Wm3=np.array([[-np.cos(np.pi/3),-np.cos(np.pi/6),0],[np.sin(np.pi/3),-np.sin(np.pi/6),0],[0,0,1]])
        return Wm3
    
        
    def calculate_P(self):
            wm2=self.get_m2_basis()
            wm3=self.get_m3_basis()
            P1=self.c1+np.array([0,self.D,0])
            P2=np.linalg.solve(wm2,self.c2)+np.array([0,self.D,0])
            P3=np.linalg.solve(wm3,self.c3)+np.array([0,self.D,0])
            #elimino la coordenana en x
            P1=np.array([P1[1],P1[2]])
            P2=np.array([P2[1],P2[2]])
            P3=np.array([P3[1],P3[2]])
            return P1,P2,P3
        
    def correct_height(self,h):
        c1x=self.c1[0]
        c2x=self.c2[0]
        c3x=self.c3[0]
        
        c1y=self.c1[1]
        c2y=self.c2[1]
        c3y=self.c3[1]
        
        c1z=self.c1[2]
        c2z=self.c2[2]
        c3z=self.c3[2]
        
        A=np.array([[c1x,c1y,1],[c2x,c2y,1],[c3x,c3y,1]])
        B=np.array([c1z,c2z,c3z]).transpose()
        params=np.linalg.solve(A,B)
        offset=np.array([0,0,h-params[2]])
        #corrijo la altura
        self.c1=self.c1+offset
        self.c2=self.c2+offset
        self.c3=self.c3+offset
        
    def solve_platform(self,r_x,r_y,h):
        if r_x==0 and r_y==0:
            self.correct_height(h)
        else:
            ang_vect=np.array([r_x,r_y,0])
            movement,joint=self.assign_movement(r_x,r_y)
            #obtengo la base más conveniente
            if joint=="C1":
                wm=np.identity(3)
            if joint=="C2":
                wm=self.get_m2_basis()
            if joint=="C3":
                wm=self.get_m3_basis()
            #referencio el vector de rotación a la base
            ang_vect=np.linalg.solve(wm,ang_vect)
            if movement=="roll":
                self.rotate(movement,joint,ang_vect[0])
            if movement=="pitch":
                self.rotate(movement,joint,ang_vect[1])
            self.correct_height(h)
        
        P1,P2,P3=self.calculate_P()
        limb_i=limb(self.l1,self.l2)
        self.ang1=limb_i.calculate_motor_angle(P1,self.max_limb_error,self.max_limb_try_count)
        self.ang2=limb_i.calculate_motor_angle(P2,self.max_limb_error,self.max_limb_try_count)
        self.ang3=limb_i.calculate_motor_angle(P3,self.max_limb_error,self.max_limb_try_count)
        #Chekeo el ángulo necesario en las juntas universales
        for joint in ["C1","C2","C3"]:
            angle=self.get_joint_angle(joint)
            if np.abs(angle)> self.max_joint_angle:
                #raise ValueError('Joint angle exceeded\n {}.angle={}º >{}º'.format(joint,angle*180/np.pi,self.max_joint_angle*180/np.pi))
                print('Joint angle exceeded\n {}.angle={}º >{}º'.format(joint,angle*180/np.pi,self.max_joint_angle*180/np.pi))
                return -1
            else:
                return 0
    def assign_movement(self,r_x,r_y):
        angle=cmath.polar(r_x+1j*r_y)[1]*(180/np.pi)
    
        if angle<0:
            angle+=180
        if angle > 165:
            return "roll","C1"
        if angle > 135 and angle <= 165:
            return "pitch","C2"
        if angle > 105 and angle <= 135:
            return "roll","C3"
        if angle > 75 and angle <= 105:
            return "pitch","C1"
        if angle <= 75 and angle >45:
            return "roll","C2"
        if angle <= 45 and angle >15:
            return "pitch","C3"  
        if angle <= 15:
            return "roll","C1"  

    def get_joint_angle(self,joint):
        "Función que obtiene el ángulo que se le exige a cada junta universal"
        "Lo que hace es tomar las distancias relativas entre cada una de las juntas"
        "y cambiar la base de los punto a una donde el eje Y atravieza el plato desde la junta analizada"
        "el eje Z permanece siendo el mismo de la base y el eje X es perpendicular a Y y Z"
        "Con las coordenadas relativas transformadas en X y Z calcula el ángulo que debe rotar la junta universal"
        "Si el plato se encuentra horizontal al piso, la función retorna 0"
        #vectores entre las juntas
        c13=self.c3-self.c1 
        c12=self.c2-self.c1
        c23=self.c3-self.c2
        c21=self.c1-self.c2
        c31=self.c1-self.c3
        c32=self.c2-self.c3
        
        #Obtengo la base de la junta en cada caso
        if joint=="C1":
            #El eje Y es la suma de las distancia C13 y C12 (vector sobre el plato que apunta hacia adelante)
            y=c13+c12
            y=y/np.linalg.norm(y)
            #El eje Z no cambia
            z=np.array([0,0,1])
            #El eje x es el producto vectorial entre el el vector "y" y el eje Z 
            x=np.cross(y,z)
            
        if joint=="C2":
            y=c23+c21
            y=y/np.linalg.norm(y)
            #El eje Z no cambia
            z=np.array([0,0,1])
            #El eje x es el producto vectorial entre el el vector "y" y el eje Z 
            x=np.cross(y,z)
        if joint=="C3":
            y=c32+c31
            y=y/np.linalg.norm(y)
            #El eje Z no cambia
            z=np.array([0,0,1])
            #El eje x es el producto vectorial entre el el vector "y" y el eje Z 
            x=np.cross(y,z)
        
        W=np.array([x,y,z]).transpose()

        
        #cambio de base los puntos
        c1_t=np.linalg.solve(W,self.c1)
        c2_t=np.linalg.solve(W,self.c2)
        c3_t=np.linalg.solve(W,self.c3)
        c13_t=c3_t-c1_t 
        c12_t=c2_t-c1_t
        c23_t=c3_t-c2_t
        c21_t=c1_t-c2_t
        c31_t=c1_t-c3_t
        c32_t=c2_t-c3_t
        if joint=="C1":
            angle=np.arctan(c13_t[2]/c13_t[0])
        if joint=="C2":
            angle=np.arctan(c23_t[2]/c23_t[0])
        if joint=="C3":
            angle=np.arctan(c32_t[2]/c32_t[0])
        return angle

        
    def generate_geogebra (self):
        s1=geogebra_script()
        #Imprimo el plato
        s1.add_point(self.c1,"C1")
        s1.add_point(self.c2,"C2")
        s1.add_point(self.c3,"C3")
        s1.define_polygon("C1","C2","C3")
        #Imprimo los planos de acción de los motores
        #s1.add_point(np.array([0,-1,0]),"AUX1")
        #s1.add_point(np.array([-np.sin(np.pi/3),np.cos(np.pi/3),0]),"AUX2")
        #s1.add_point(np.array([np.sin(np.pi/3),np.cos(np.pi/3),0]),"AUX3") 
        #s1.add_point(np.array([0,0,0]),"AUX0") 
        #s1.add_point(np.array([0,0,1]),"AUX4")
        #s1.define_plane("AUX1","AUX0","AUX4")
        #s1.define_plane("AUX2","AUX0","AUX4")
        #s1.define_plane("AUX3","AUX0","AUX4")
        #imprimo los ejes de los motores
        D1=self.D*np.array([0,-1,0])
        D2=self.D*np.array([-1*np.sin(np.pi/3),np.cos(np.pi/3),0])
        D3=self.D*np.array([np.sin(np.pi/3),np.cos(np.pi/3),0])
        s1.add_point(D1,"M1")
        s1.add_point(D2,"M2")
        s1.add_point(D3,"M3")
        #imprimo el brazo l1
        a1=np.array([0,np.cos(self.ang1),np.sin(self.ang1)])*self.l1 #esto es visto desde el plano de referencia del motor
        a1=a1-np.array([0,self.D,0])#muevo el cero al centro de la plataforma
        s1.add_point(a1,"A1")
        s1.define_segment("M1","A1")
        s1.define_segment("A1","C1")

        #imprimo el brazo l2
        a2=np.array([0,np.cos(self.ang2),np.sin(self.ang2)])*self.l1  #esto es visto desde el plano de referencia del motor
        a2=a2-np.array([0,self.D,0])#muevo el cero al centro de la plataforma
        wm2=self.get_m2_basis()
        a2=wm2.dot(a2)
        s1.add_point(a2,"A2")
        s1.define_segment("M2","A2")
        s1.define_segment("A2","C2")
        #imprimo el brazo l3
        a3=np.array([0,np.cos(self.ang3),np.sin(self.ang3)])*self.l1  #esto es visto desde el plano de referencia del motor
        a3=a3-np.array([0,self.D,0])#muevo el cero al centro de la plataforma
        wm3=self.get_m3_basis()
        a3=wm3.dot(a3)
        s1.add_point(a3,"A3")
        s1.define_segment("M3","A3")
        s1.define_segment("A3","C3")        
        s1.print_script()       

In [276]:
class geogebra_script:
    def __init__(self):
        self.script=""
        self.first_item=1
        
    def add_point(self,point,name):
        if self.first_item==0:
            self.script+=","
        self.first_item=0
        self.script+="\"{0}=({1:.3f},{2:.3f},{3:.3f})\"\n".format(name,point[0],point[1],point[2])

    def add_2D_point(self,point,name):
        if self.first_item==0:
            self.script+=","
        self.first_item=0
        self.script+="\"{0}=({1:.3f},{2:.3f})\"\n".format(name,point[0],point[1])
        
    def define_polygon(self,point1,point2,point3):
        if self.first_item==0:
            self.script+=","
        self.first_item=0
        self.script+="\""
        self.script+="Polygon({},{}, {})".format(point1,point2,point3)
        self.script+="\""
        self.script+="\n"

    def define_plane(self,point1,point2,point3):
        if self.first_item==0:
            self.script+=","
        self.first_item=0
        self.script+="\""
        self.script+="Plane({},{}, {})".format(point1,point2,point3)
        self.script+="\""
        self.script+="\n"
    def define_segment(self,point1,point2):
        if self.first_item==0:
            self.script+=","
        self.first_item=0
        self.script+="\""
        self.script+="Segment({},{})".format(point1,point2)
        self.script+="\""
        self.script+="\n"   
    def print_script(self):
        print("Execute[{\n"+self.script+"}]")

In [277]:
class limb:
    def __init__(self,l1,l2):
        self.l1=l1
        self.l2=l2
    def evaluate_angle(self,P,angle):
        a=self.l1*np.array([np.cos(angle),np.sin(angle)])
        b=((P-a)/(np.linalg.norm(P-a)))*self.l2
        return np.linalg.norm(P-a)-self.l2
     
    def calculate_motor_angle(self,P,max_error,tries):
        P_angle=cmath.polar(P[0]+1j*P[1])[1]
        angle_min=-90*np.pi/180
        angle_max=P_angle
        try_count=0
        while(try_count<tries):
            angle=0.5*(angle_max+angle_min)
            e=self.evaluate_angle(P,angle)
            if np.abs(e)<max_error:
                break
            if e>0:
                angle_min=angle
            if e<0:
                angle_max=angle
            try_count+=1
        if try_count==tries:
            #raise ValueError('Point unreachable\nP=({},{})'.format(P[0],P[1]))
            print('Point unreachable\nP=({},{})'.format(P[0],P[1]))
        else:
            return angle

In [278]:
p1=platform(d=15,D=20,l1=6,l2=8,max_limb_error=0.0001,max_limb_try_count=30,max_joint_angle=30*np.pi/180)

#p1.generate_geogebra()

In [283]:
angle_spacex=np.linspace(start=-20*np.pi/180,stop=20*np.pi/180,num=10)
angle_spacey=np.linspace(start=-20*np.pi/180,stop=20*np.pi/180,num=10)
s1=geogebra_script()
point=0
for angle_x in angle_spacex:
    for angle_y in angle_spacey:
        p1=platform(d=15,D=20,l1=6,l2=8,max_limb_error=0.0001,max_limb_try_count=30,max_joint_angle=25*np.pi/180)
        result=p1.solve_platform(r_x=angle_x,r_y=angle_y,h=5) 
        if result==-1:
            print("angle exceeded")
            print("angle_x={}".format(angle_x*180/np.pi))
            print("angle_y={}".format(angle_y*180/np.pi))
        else:
            P1,P2,P3=p1.calculate_P()
            name="P"+str(point)
            s1.add_2D_point(P1,name)
            point=point+1

Point unreachable
P=(5.0,15.06248747066789)
Point unreachable
P=(5.0000000000000036,15.062487470667884)
Point unreachable
P=(14.053834123762972,-2.977762617086407)
Joint angle exceeded
 C1.angle=-49.28315172726175º >25.0º
angle exceeded
angle_x=-50.0
angle_y=-20.0
Point unreachable
P=(5.0,14.287900049750789)
Point unreachable
P=(5.0000000000000036,14.287900049750785)
Point unreachable
P=(13.364428884419409,-3.217402839270118)
Joint angle exceeded
 C1.angle=-47.00281427391539º >25.0º
angle exceeded
angle_x=-50.0
angle_y=-15.555555555555554
Point unreachable
P=(2.3209070726490424,5.0)
Point unreachable
P=(10.358185854701912,-4.951209222534075)
Point unreachable
P=(10.358185854701912,14.951209222534075)
Joint angle exceeded
 C1.angle=50.0º >25.0º
angle exceeded
angle_x=-50.0
angle_y=-11.111111111111112
Point unreachable
P=(2.3209070726490424,5.0)
Point unreachable
P=(10.358185854701912,-4.951209222534075)
Point unreachable
P=(10.358185854701912,14.951209222534075)
Joint angle exceeded
 C1

Point unreachable
P=(6.179781666779181,10.049868702905089)
Point unreachable
P=(5.819529524134712,0.7649447686795501)
Point unreachable
P=(4.590235237932649,5.000000000000001)
Point unreachable
P=(5.819529524134712,9.235055231320448)
Point unreachable
P=(5.0,7.0346212880150905)
Point unreachable
P=(5.0000000000000036,7.034621288015089)
Point unreachable
P=(5.78487149644349,1.143679591305844)
Point unreachable
P=(5.0,6.7254235814987435)
Point unreachable
P=(5.0000000000000036,6.725423581498742)
Point unreachable
P=(5.572778853112748,1.6809243223551191)
Point unreachable
P=(4.859414829705383,5.0)
Point unreachable
P=(5.281170340589231,2.4965943011694303)
Point unreachable
P=(5.281170340589231,7.503405698830569)
Point unreachable
P=(4.859414829705383,5.0)
Point unreachable
P=(5.281170340589231,2.4965943011694303)
Point unreachable
P=(5.281170340589231,7.503405698830569)
Point unreachable
P=(4.999999999999998,3.2745764185012556)
Point unreachable
P=(5.572778853112753,8.319075677644879)
Poi

In [280]:
s1.print_script()

Execute[{
"P0=(5.000,8.874)"
,"P1=(5.000,8.513)"
,"P2=(5.000,8.164)"
,"P3=(5.000,7.827)"
,"P4=(4.548,5.000)"
,"P5=(4.548,5.000)"
,"P6=(5.000,2.173)"
,"P7=(5.000,1.836)"
,"P8=(5.000,1.487)"
,"P9=(6.673,10.962)"
,"P10=(6.416,-0.510)"
,"P11=(5.000,7.916)"
,"P12=(5.000,7.586)"
,"P13=(5.000,7.265)"
,"P14=(4.725,5.000)"
,"P15=(4.725,5.000)"
,"P16=(5.000,2.735)"
,"P17=(5.000,2.414)"
,"P18=(6.020,9.708)"
,"P19=(6.416,10.510)"
,"P20=(6.180,-0.050)"
,"P21=(5.820,0.765)"
,"P22=(5.000,7.035)"
,"P23=(5.000,6.725)"
,"P24=(4.859,5.000)"
,"P25=(4.859,5.000)"
,"P26=(5.000,3.275)"
,"P27=(5.523,8.401)"
,"P28=(5.820,9.235)"
,"P29=(6.180,10.050)"
,"P30=(5.964,0.418)"
,"P31=(5.641,1.244)"
,"P32=(5.382,2.088)"
,"P33=(5.000,6.202)"
,"P34=(5.000,5.906)"
,"P35=(5.000,4.094)"
,"P36=(5.000,3.798)"
,"P37=(5.382,7.912)"
,"P38=(5.641,8.756)"
,"P39=(5.964,9.582)"
,"P40=(6.357,9.966)"
,"P41=(5.824,8.946)"
,"P42=(5.422,7.863)"
,"P43=(5.108,3.443)"
,"P44=(5.000,5.398)"
,"P45=(5.021,5.688)"
,"P46=(5.108,6.557)"
,"P47=(5.

In [281]:
p1.solve_platform(r_x=-20.0*np.pi/180,r_y=-20.0*np.pi/180,h=5)

0